In [1]:
#r "C:\BoSSS\experimental\public\src\L4-application\PublicTestRunner\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [1]:
using System.Diagnostics;
using BoSSS.Solution.NSECommon;
using static System.Math;
using BoSSS.Solution.Queries;
using BoSSS.Application.LowMachCombustionNSE;
using System.IO;
using System.Globalization;

In [1]:
string WFlowName = "HeatedSquareCavityConvStudy"; 
WorkflowMgm.Init(WFlowName);

In [1]:
myDb.Sessions

In [1]:
var myBatch = new MiniBatchProcessorClient(@"C:\deploy-Mini");
MiniBatchProcessor.Server.StartIfNotRunning();

In [1]:
string dbPath = @"C:\Databases\HeatedSquareCavity_NusseltStudy";
var myDb      = OpenOrCreateDatabase(dbPath);

In [1]:
Type solver = typeof(BoSSS.Application.LowMachCombustionNSE.LowMachCombustionNSEMain);

In [1]:
// ==========
// Parameters
// ==========
int[] dgDegree = new int[]{1,2,3,4};//
int[] Resolutions = new int[]{2,3,4,5,6,7};//
//int[] dgDegree = new int[]{1,2};
//int[] Resolutions = new int[]{3,4};

double[] RaArr = new double[]{1e2,1e4,1e6};
double[] newtonConvCriteria = new double[] {1e-10}; // 1e-5, 1e-7, 1e-9,
double dt = -1;
var  NofCells = Resolutions.ToList().Select(r=> (int)Math.Pow(2,r)).ToArray();
string baseJobName = "HeatedSquareCavity"   

In [1]:
NofCells

In [1]:
for(int n = 0; n < RaArr.Length; n++){
for (int dg = 0; dg < dgDegree.Length; dg++){
        for(int r = 0; r < NofCells.Length; r++){
        string ending = "P" + dgDegree[dg] + "K" +NofCells[r]+"Ra"+n;
         string jobName       = string.Format(baseJobName + ending);
         string StartupString = string.Format("cs:BoSSS.Application.LowMachCombustionNSE.FullNSEControlExamples.NaturalConvectionSquareCavity({0}, {1},{2}, {3}, @\"{4}\" )", dgDegree[dg], NofCells[r] ,RaArr[n], dt, dbPath);
Console.WriteLine(StartupString);
                var oneJob = new Job(jobName, solver);
          oneJob.ExecutionTime = "0:15:00"; // 
        oneJob.SetControlStatement(StartupString);
         oneJob.NumberOfMPIProcs = NofCells[r] < 16? 1:6 ;//StaticFormulas.getNumberOfcores(NofCells[r]*NofCells[r],dgDegree[dg],4);
         oneJob.NumberOfThreads = 1;
         oneJob.Activate(myBatch);
    }
}
}

In [1]:
Dictionary<int,double[]> All_p0values = new Dictionary<int,double[]>();

In [1]:
foreach (int pDeg in dgDegree) {
    double[] p0values;
    var pDegSessions = myDb.Sessions.Where(
         Si => (Convert.ToInt32(Si.KeysAndQueries["DGdegree:Velocity*"]) == pDeg) // function which is true on all sessions we are interested in: 
      ).ToArray();

    p0values = new double[pDegSessions.Length];
    int i = 0;
    foreach (var sess in pDegSessions) {
        double p0value = sess.Timesteps.Last().Fields.Pick(0).ProbeAt(0, 0);
        p0values[i] = p0value;
        i++;
    }
All_p0values.Add(pDeg,p0values);
}

In [1]:
string basepath = @"C:\tmp\NaturalConvection\data\ThermoPress\";
foreach (int dgDg in dgDegree) {
    if (dgDg != 5) {
        string names = "p0" + dgDg.ToString() + ".txt";
        string txtpath = basepath + names;
        using (StreamWriter stw = new StreamWriter(txtpath)) {
            stw.WriteLine("x\ty");
            for (int i = 0; i < Resolutions.Length; i++) {
                stw.Write(Resolutions[i].ToString("E16", NumberFormatInfo.InvariantInfo) + "\t");
                stw.Write(All_p0values[dgDg][i].ToString("E16", NumberFormatInfo.InvariantInfo));
                stw.WriteLine();
            }
        }
    }
}

In [1]:
string basepath = @"C:\Users\jfgj8\Dropbox\PaperdiffusionFlame\ama\data\NusseltStudy\";
char ReChar     = '2';

In [1]:
 Dictionary<int,double[]> All_NULeftvalues = new Dictionary<int,double[]>();
Dictionary<int,double[]> All_NURightvalues = new Dictionary<int,double[]>();
Dictionary<int,double[]> All_NUTopvalues = new Dictionary<int,double[]>();
 foreach (int pDeg in dgDegree) {
    var pDegSessions = myDb.Sessions.Where(
         Si => ((Convert.ToInt32(Si.KeysAndQueries["DGdegree:Velocity*"]) == pDeg) &&
          (((Si.KeysAndQueries["SessionName"]).ToString()).Last() == ReChar)
       // &&(Convert.ToInt32(Si.KeysAndQueries["Grid:NoOfCells"])) > 64
        ) // function which is true on all sessions we are interested in: 
      ).ToArray();

    double[] leftvalues = new double[pDegSessions.Length];
    double[] rightvalues = new double[pDegSessions.Length];
    int i = 0;
    foreach (var sess in pDegSessions) {
        var leftvalue  = sess.KeysAndQueries["NusseltLeft"];
        var rightvalue = sess.KeysAndQueries["NusseltRigth"];
        var numofcells = sess.KeysAndQueries["Grid:NoOfCells"];
        leftvalues[i] = System.Convert.ToDouble(leftvalue);
        rightvalues[i] = System.Convert.ToDouble(rightvalue);
rightvalues[i]*=-1;
        i++;
    }
All_NULeftvalues.Add(pDeg,leftvalues);
All_NURightvalues.Add(pDeg,rightvalues);
}


In [1]:
foreach (int dgDg in dgDegree) {
    string names   = "NusseltReLeft" + ReChar +"Dg" + dgDg.ToString() + ".txt";
    string txtpath = basepath + names;
    using (StreamWriter stw = new StreamWriter(txtpath)) {
        stw.WriteLine("x\ty");
        for (int i = 0; i < 4; i++) {
            stw.Write(NofCellsInv[i].ToString("E16", NumberFormatInfo.InvariantInfo) + "\t");
            stw.Write(All_NULeftvalues[dgDg][i].ToString("E16", NumberFormatInfo.InvariantInfo));
            stw.WriteLine();
        }
    }
}

In [1]:
foreach (int dgDg in dgDegree) {
    string names   = "NusseltReRight" + ReChar +"Dg" + dgDg.ToString() + ".txt";
    string txtpath = basepath + names;
    using (StreamWriter stw = new StreamWriter(txtpath)) {
        stw.WriteLine("x\ty");
        for (int i = 0; i < 4; i++) {
            stw.Write(NofCellsInv[i].ToString("E16", NumberFormatInfo.InvariantInfo) + "\t");
            stw.Write(All_NURightvalues[dgDg][i].ToString("E16", NumberFormatInfo.InvariantInfo));
            stw.WriteLine();
        }
    }
}

In [1]:
NofCellsInv

In [1]:
myDb.Sessions.Pick(34).KeysAndQueries